In [367]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
from pkg_resources import ContextualVersionConflict

# from rich.console import Console

import requests
import json

import time
from sqlalchemy import delete
from tqdm import tqdm
import re

# console = Console()

sample_transport=RequestsHTTPTransport(
    url='http://localhost:4001/graphql',
    use_json=True,
    headers={
        "Content-type": "application/json",
    },
    verify=False
)

client = Client(
    #execute_timeout=20,
    transport=sample_transport,
    fetch_schema_from_transport=True,
)

session = requests.Session()
session.auth = ('admin', "secret")

#hostname = 'http://ec2-3-64-237-95.eu-central-1.compute.amazonaws.com:8080/tilt/tilt'
#auth = session.post(hostname)
#response = json.loads(session.get(hostname).content)

#tilt = json.loads(requests.get('http://ec2-3-64-237-95.eu-central-1.compute.amazonaws.com:8080/tilt/tilt').content)
#print("Fetch Successful!", response)


json_file = open('/Users/johannes/Desktop/TU/playground/tilt')
tilt = json.load(json_file)

In [412]:
hostname = 'https://ec2-3-64-237-95.eu-central-1.compute.amazonaws.com:8081/db/tilt/tilt'
auth = session.post(hostname)
response = json.loads(session.get(hostname).content)

ConnectionError: HTTPConnectionPool(host='ec2-3-64-237-95.eu-central-1.compute.amazonaws.com', port=8080): Max retries exceeded with url: /tilt/tilt (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fe5d1efa3d0>: Failed to establish a new connection: [Errno 60] Operation timed out'))

In [406]:

def create_node(parent):

    """
    func:   create_node
            Recursively goes through JSON and builds a mutation to generate the neo4j graph
            Base case is when an entry is a string
    """

    def rem_dq(uncleaned):

        """
        func:   rem_dq
                removes double quotes. 
                Needed to avoid errors in upload
        """
        if isinstance(uncleaned,str):
            return re.sub('"', '', uncleaned)
        else:
            return uncleaned

    #mutation string
    label_string = ""

    #iterate through elements of dictionary or list
    for i, key in enumerate(parent):

        #if key == 'rightToRectificationOrDeletion':
            #label_string += " HERE "
            
        try:
            if key[0] == "$": # the dollar sign infront of the $oid is not recognised so we have to remove it
                label_string += f'{key[1:]}: "{rem_dq(parent[key])}"'


            # if key is a list
            elif isinstance(parent[key], list):

            
                #if list item is empty
                if len(parent[key]) == 0:
                    
                    #label_string += f'{key}: "{parent[key]}"'
                    label_string += "" # just add nothing. not ideal as we lose out on recording missing information but easier to fix than trying to fix missfilled fields in tilt

                #two types of entries: nodes or strings
                else:

                    # When we get lists with multiple same node entries, we make a list of nodes. 
                    # There we can only need one create statement and then close it afterwards
                    label_string += key+ ':{create:['
                    node_list = True

                    for list_item in parent[key]:
                        
                        # some entries are a string wrapped in a lists instead of a node.
                        if type(list_item) == type(""): 
                            #deleted = label_string[-len(f'{key}:{{create:['):]
                            label_string = label_string[:-len(f'{key}:{{create:[')]                             
                            label_string += f'{key}: "{rem_dq(list_item)}"'
                            node_list = False

                        # if the list items are dictionaries make a new node
                        else: 
                            label_string += f'{{{create_node(list_item)}}},'

                    #
                    if node_list:
                        label_string = label_string[:-1]+']}'
            
                
            # if the element is a dictionary
            elif isinstance(parent[key], dict): 
                
                #create the next node for this dict
                label_string += f'{key}:{{create:[{{{create_node(parent[key])}}}]}}'

            else: # base case
                
                if i < len(parent)-1:

                    label_string += f'{key}: "{rem_dq(parent[key])}", '
                    
                else:
                    label_string += f'{key}: "{rem_dq(parent[key])}"'
                
        except Exception as ex:
            label_string += "BIG ERROR HERE"  
            print(ex)  

    #certain characters (like tilda) are rejected by gql
    label_string = re.sub('~', "", label_string)
    label_string = label_string.strip().replace('\n', "")
    
    return label_string

    

In [407]:
print("mutation { createtilts( input: [ { " + create_node(tilt[2]) + "}] ) {tilts {meta { name } } }}")

mutation { createtilts( input: [ { _id:{create:[{oid: "62559206e7dfaf542de42c60"}]}meta:{create:[{_id: "f364a1b7-af84-43ae-a18d-7aa9b0c2f259", name: "Amazon Alexa Terms of Use", created: "2021-07-16T08:44:48.404159", modified: "2021-07-16T08:44:48.404180", version: "1", language: "en", status: "active", url: "https://www.amazon.com/gp/help/customer/display.html?nodeId=201809740", _hash: "2ebec9c58a92f02efa1906cf685d13c67331008c3ca47428b431c3b3b157c47b"}]}controller:{create:[{name: "Amazon.com Services LLC", country: "None", address: "None", division: "None", representative:{create:[{name: "None", email: "None", phone: "None"}]}}]}dataProtectionOfficer:{create:[{email: "None", phone: "None", country: "None", name: "None", address: "None"}]}dataDisclosed:{create:[{_id: "277eb7ac-b510-4e9c-92e0-7228c3371461", category: "your use of Alexa, your Alexa Interactions, and your Alexa Enabled Products and Auxiliary Products", purposes:{create:[{}]}legalBases:{create:[{}]}legitimateInterests:{cre

In [408]:
tilt_mutations = []
for tilt_entry in tqdm(tilt):
    tilt_entry  
    tilt_mutations.append("mutation { createtilts( input: [ { " + create_node(tilt_entry) + "}] ) {tilts {meta { name } } }}")

100%|██████████| 73/73 [00:00<00:00, 1877.58it/s]


In [411]:
successes = 0
for i, query_string in enumerate(tqdm(tilt_mutations)):
    #print("Query String: ", query_string)
    try:
        query = gql(query_string)
        client.execute(query)
        successes += 1
    except Exception as syn:
        print(syn)
        print("Error With Tilt No.", i)

print("Successfully Uploaded:", 100*(successes/(i+1)), "per. of tilts")

100%|██████████| 73/73 [01:41<00:00,  1.39s/it]

Successfully Uploaded: 100.0 per. of tilts
